# NCSU GIS 582: Geospatial Modeling and Analysis

## Introduction to Jupyter Notebooks Tutorial B

### Getting Started with GRASS

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

This notebook is based on sections of [GIS-based Analysis of Coastal Lidar Time-Series by Hardin et al (2014)](https://link.springer.com/book/10.1007/978-1-4939-1835-5).

In this notebook we will:
* [Import Python and GRASS Python API packages](#1.-Import-Python-Packages)
* [Create a new GRASS project](#2.-Create-a-New-Project)
* [Import data](#4.-Import-Data)
* Get a quick overview of [the Python API](#6.-GRASS-Python-API)
* [Visualize data](#6.-Data-Visualization-with-grass.jupyter)

## Google Colab Setup

Let’s first print system description to know where are we.

In [ ]:
!lsb_release -a

At the time of writing this tutorial, Colab has Linux Ubuntu 22.04.4 LTS. So we add the ppa:ubuntugis repository, update and install GRASS. It might take a couple of minutes according to the resources available.

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev

Check that GRASS is installed by asking which version is there.

In [ ]:
!grass --version

***

## 1. Import Python Packages

Import the Python standard libraries we need.

In [ ]:
import subprocess
import sys
import os
from pathlib import Path

We are going to import a GRASS Python API (`grass.tools`) and the GRASS GIS Jupyter package (`grass.jupyter`), but first, we need to find the path to those packages using the `--config python_path` command. This command is slightly different for each operating system.

We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

And now we can import the GRASS python packages!

In [ ]:
# Import the GRASS GIS packages we need.
import grass.tools as Tools
import grass.jupyter as gj

***

## 2. Create a New Project

In [ ]:
import grass.script as gs
gs.create_project("nags_head", epsg=3358, overwrite=True)

***

## 3. Start GRASS Session

In [ ]:
gj.init("nags_head")

We've launched GRASS now! We can access GRASS commands using the command line interface (with the `!` line magic):

In [ ]:
!g.version

In [ ]:
!g.region -p

***

## 4. Import Data

For this tutorial, we will import a NAIP image of Nags Head.

In [ ]:
!r.import input="./naip_2020.tif" output="naip_2020" resolution=value resolution_value=1

Look at the computational region above. Since it's never been set before, it defaults to an area that's 1m by 1m! Let's change the computation region to match the imagery we just imported. How many cells are in the computation area now?

In [ ]:
!g.region raster="naip_2020.1" -p

GRASS functionality is available through tools (also called modules). There are over 500 different tools in the core distribution and over 300 addon tools or extensions that can be used to prepare and analyze data.

Tools respect the following naming conventions:

Prefix | Function | Example
------ | -------- | -------
r.* | raster processing | r.mapcalc: map algebra
v.*	| vector processing	| v.clean: topological cleaning
i.*	| imagery processing | i.segment: object recognition
db.* | database management | db.select: select values from table
r3.* | 3D raster processing | r3.stats: 3D raster statistics
t.* | temporal data processing | t.rast.aggregate: temporal aggregation
g.* | general data management | g.rename: renames map
d.* | display | d.rast: display raster map

Note also that some tools have multiple dots in their names. For example, tools staring with v.net.* deal with vector network analysis and r.in.* tools import raster data into GRASS GIS spatial database.

There is also a tool for finding other tools:

In [ ]:
!g.search.modules keyword=zonal

Here is how to get all options and flags of a GRASS tool through command line:

In [ ]:
!r.univar --help

For the online version of manual pages, you might visit: <https://grass.osgeo.org/grass-stable/manuals/index.html> 

***

## 5. GRASS Python API

There are multiple [Python APIs](https://grass.osgeo.org/grass85/manuals/python_intro.html) for accessing GRASS tools' functionality: `grass.tools` provides a Python interface to GRASS tools, `grass.script` handles GRASS projects and sessions in Python, and `grass.pygrass` enables a fine-grained access to the GRASS data structures. PyGRASS is advantageous for more advanced workflows and low level tasks.

Here, we will be using `grass.tools` as it is simple and straightforward to use.
 

In [ ]:
tools = Tools()

Tools can be accessed as methods of the Tools object. The method names use underscores (snakecase) instead of dots (periods). For example, when listing the data available in a project with `g.list`, we call the `g_list` method and ask for the returned text:

In [ ]:
tools.g_list(type="raster").text

**Try it yourself!**

_The `r.info map=NAME` command will print information about the raster NAME. Execute `r.info` in Python to get information about `'naip_2020.1'`._

<details>
    <summary>👉 <b>click to see an example</b></summary>
    
```python
tools.r_info(map="naip_2020.1", format=plain).text
```
</details>

***

## 6. Data Visualization with `grass.jupyter`

`grass.jupyter.Map()` creates and displays GRASS maps as PNG images. `gj.Map()` accepts any GRASS display module as a method by replacing the "." with "\_" in the module name. For example:

In [ ]:
example = gj.Map()
example.d_rast(map="naip_2020.1", flags="i") # d.rast map=naip_2020.1
example.d_barscale(bgcolor="none") # d.barscale
example.show()

To display the image, we call the `show()` method. You can also save the image with the `save()` method.

We also might want to make this a nice square. Instead of clipping the image, we can adjust the computational region.

Here we use the `grow` parameter with a negative value to shrink the region by 100 m on each side. See the [`g.region`](https://grass.osgeo.org/grass-stable/manuals/g.region.html) manual page for more options.

In [ ]:
!g.region grow=-100 -p

Now, it's a nice rectangle!

In [ ]:
example = gj.Map(use_region=True)
example.d_rast(map="naip_2020.1") # d.rast map=naip_2020.1
example.d_barscale() # d.barscale
example.show()

Let's save the region so we can use it later.

In [ ]:
!g.region --help

In [ ]:
!g.region save=jockeys_ridge

And now we're off and running with GRASS GIS in Jupyter Notebooks!

In this tutorial, we learned:
1. How to launch a GRASS GIS project in a Jupyter Notebook (including importing data and managing computational region)
2. How to execute GRASS GIS commands using the command line interface and with the Python API
3. How to create basic visualizations of GRASS GIS data

## Acknowledgements

These materials were created with support from the National Science Foundation (Award [2303651](https://www.nsf.gov/awardsearch/showAward?AWD_ID=2303651)), the Digital Education and Learning Technologies Applications (DELTA) Center at NC State University and the [Center for Geospatial Analytics](https://cnr.ncsu.edu/geospatial/) at NC State.